# 1. Logits and Greedy Decoding

In this notebook, we'll strip away the magic of `model.generate()` and look at exactly what an LLM outputs given a prompt. Then, we will implement the simplest text generation strategy: **Greedy Decoding**.

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set up device (MPS is for Apple Silicon Macs, CUDA for Nvidia, otherwise CPU)
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# We'll use Qwen/Qwen2.5-0.5B for this default as it is fast and easy to understand.
# Other great tiny models you can swap here:
# - "HuggingFaceTB/SmolLM2-135M-Instruct"
# - "gpt 2"
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

model.eval()  # Make sure we're in eval mode (disables dropout, etc.)
print(f"{model_id} Model and tokenizer loaded!")

Using device: mps


Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Qwen/Qwen2.5-0.5B Model and tokenizer loaded!


## Step 1: The Forward Pass and Logits
When you pass tokens into the model, it outputs `logits`. These are raw, un-normalized scores for every single token in the vocabulary, representing how strongly the model believes that token comes next.

In [2]:
prompt = "The quick brown fox jumps over the lazy"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

print("Input tokens (IDs):", inputs["input_ids"])
print("Decoded inputs:", [tokenizer.decode(t) for t in inputs["input_ids"][0]])

# Run the forward pass
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
print(f"\nLogits shape: {logits.shape}")
print("Shape meaning: (batch_size, sequence_length, vocab_size)")

Input tokens (IDs): tensor([[  785,  3974, 13876, 38835, 34208,   916,   279, 15678]],
       device='mps:0')
Decoded inputs: ['The', ' quick', ' brown', ' fox', ' jumps', ' over', ' the', ' lazy']

Logits shape: torch.Size([1, 8, 151936])
Shape meaning: (batch_size, sequence_length, vocab_size)


The logits shape tells us we have a score for *every* position in the input sequence. We only care about predicting the *next* word, so we want the logits for the **very last position** in the sequence.

In [3]:
# Get logits for the last token in the sequence
next_token_logits = logits[0, -1, :]
print(f"Next token logits shape: {next_token_logits.shape} (One score for every token in vocabulary)")

# Convert raw logits into probabilities between 0 and 1 using Softmax
probabilities = F.softmax(next_token_logits, dim=-1)

# Let's look at the top 5 most likely next tokens
top_k = 5
top_probs, top_indices = torch.topk(probabilities, top_k)

print("\nTop 5 predictions for the next token:")
for i in range(top_k):
    token_id = top_indices[i].item()
    prob = top_probs[i].item()
    token_str = tokenizer.decode(token_id)
    print(f"{i+1}. '{token_str}' (Token ID: {token_id}) - Probability: {prob:.4f}")

Next token logits shape: torch.Size([151936]) (One score for every token in vocabulary)

Top 5 predictions for the next token:
1. ' dog' (Token ID: 5562) - Probability: 0.8789
2. '
' (Token ID: 198) - Probability: 0.0171
3. ' dogs' (Token ID: 12590) - Probability: 0.0118
4. ' brown' (Token ID: 13876) - Probability: 0.0092
5. ' (' (Token ID: 320) - Probability: 0.0086


## Step 2: Implementing a Greedy Decoding Loop
Greedy decoding means we *always* pick the token with the highest probability (the `argmax`). 

Let's build a loop to generate a whole sentence manually by constantly feeding our new tokens back into the model.

In [4]:
def greedy_generate(prompt, max_new_tokens=20):
    # Tokenize the input prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    print(f"Prompt: {prompt}")
    print("Generating: ", end="")
    
    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids)
        
        # 1. Get logits of the very last token
        next_token_logits = outputs.logits[0, -1, :]
        
        # 2. Choose the one with the highest probability (argmax)
        next_token_id = torch.argmax(next_token_logits).unsqueeze(0).unsqueeze(0)
        
        # 3. Decode and print it out as we go
        next_word = tokenizer.decode(next_token_id[0])
        print(next_word, end="", flush=True)
        
        # 4. Append token to our input IDs and repeat the loop!
        input_ids = torch.cat([input_ids, next_token_id], dim=-1)
        
    print("\n\nDone!")
    return tokenizer.decode(input_ids[0])

# Try it out! Notice how plain greedy decoding can sometimes get stuck in repetitive loops.
generated_text = greedy_generate("Artificial intelligence is going to", max_new_tokens=40)

Prompt: Artificial intelligence is going to
Generating:  be a big part of the future of the world. It’s already being used in many different ways, from self-driving cars to chatbots to virtual assistants. But what exactly is AI and how can

Done!


## 🔬 Experimentation Ideas

1. **Change the prompt:** from factual to creative (e.g., “The theory of relativity states…” vs “Once upon a time…”).
   * *Does greedy decoding behave differently?*
2. **Print the top 10 tokens and their probabilities:** at each step.
   * *Is the highest probability much larger than the others?*
3. **Measure entropy of the next-token distribution:** (using the `metrics.py` module).
   * *Does entropy decrease as the sentence progresses?*
4. **Force generation to continue for 100+ tokens:**
   * *Observe repetition or looping behavior.*
5. **Compare your manual loop with `model.generate()` using greedy mode:**
   * *Are they identical?*

In [10]:
import sys
import os
sys.path.append(os.path.abspath('.'))
from metrics import calculate_entropy

print("=== Experiment 1 & 4: Creative Prompt vs Factual Prompt (100 tokens) ===")
creative_prompt = "Once upon a time in a magical"
factual_prompt = "The theory of relativity states that"

print("\n--- Creative ---")
creative_output = greedy_generate(creative_prompt, max_new_tokens=100)
print("\n--- Factual ---")
factual_output = greedy_generate(factual_prompt, max_new_tokens=100)

print("\n=== Experiment 2 & 3: Top 10 Tokens, Probabilities, and Entropy ===")
# We will just run 5 steps to see how the distribution changes
test_prompt = "The theory of relativity states that"
input_ids = tokenizer(test_prompt, return_tensors="pt").input_ids.to(device)

for step in range(5):
    with torch.no_grad():
        outputs = model(input_ids)
    
    logits = outputs.logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    entropy = calculate_entropy(probs)
    print(f"\nStep {step+1} Entropy: {entropy:.4f}")
    
    top_probs, top_indices = torch.topk(probs, 10)
    for i in range(10):
        token_str = repr(tokenizer.decode(top_indices[i].item()))
        print(f"  {i+1}. {token_str}: {top_probs[i].item():.4f}")
        
    # Append the greedy choice to continue
    next_token = torch.argmax(logits).unsqueeze(0).unsqueeze(0)
    input_ids = torch.cat([input_ids, next_token], dim=-1)

print("\n=== Experiment 5: Compare with model.generate() ===")
inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

# Greedy by default (do_sample=False)
outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False, pad_token_id=tokenizer.eos_token_id)
hf_generated = tokenizer.decode(outputs[0])
print(f"\nHuggingFace default generate(): {hf_generated}")


=== Experiment 1 & 4: Creative Prompt vs Factual Prompt (100 tokens) ===

--- Creative ---
Prompt: Once upon a time in a magical
Generating:  kingdom, there lived a wise wizard named Merlin. He was known for his ability to predict the weather with incredible accuracy. One day, he decided to test his prediction skills by predicting the weather for the next 10 days. He used a special formula to calculate the probability of rain for each day, based on the weather patterns of the previous 10 days. The formula was as follows:

\[ P(\text{Rain on day } n) = \frac{1}{n+1

Done!

--- Factual ---
Prompt: The theory of relativity states that
Generating:  the speed of light is constant in all inertial frames of reference. This means that if you are moving at a constant speed, the speed of light will appear to slow down for you. This is known as the Lorentz transformation. The Lorentz transformation is a set of equations that describe how the speed of light changes in different reference frames. I

In [9]:
import sys
import os
sys.path.append(os.path.abspath('.'))
from metrics import calculate_entropy

# A prompt designed to trap greedy decoding into a loop
loop_prompt = "A cat is a cat is a"
input_ids = tokenizer(loop_prompt, return_tensors="pt").input_ids.to(device)

print(f"--- The Greedy Trap ---")
print(f"Prompt: {repr(loop_prompt)}\n")
print("Tracking the next 5 tokens, their probabilities, and the distribution entropy:")

for step in range(5):
    with torch.no_grad():
        outputs = model(input_ids)
    
    logits = outputs.logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    entropy = calculate_entropy(probs)
    
    # We want to see the top 3 choices to understand how the model is narrowing down
    top_probs, top_indices = torch.topk(probs, 3)
    
    print(f"\n[Step {step+1}] Entropy: {entropy:.4f}")
    for i in range(3):
        token_str = repr(tokenizer.decode(top_indices[i].item()))
        print(f"  {i+1}. {token_str}: {top_probs[i].item():.4f} " + ("<-- Greedy Choice" if i == 0 else ""))
        
    # Append the greedy choice to continue the loop
    next_token = top_indices[0].unsqueeze(0).unsqueeze(0)
    input_ids = torch.cat([input_ids, next_token], dim=-1)

print("\n--- Generating 30 more tokens to see the full loop ---")
print(tokenizer.decode(input_ids[0]), end="") # Print what we have so far
for _ in range(30):
    with torch.no_grad():
        outputs = model(input_ids)
    next_token = torch.argmax(outputs.logits[0, -1, :]).unsqueeze(0).unsqueeze(0)
    print(tokenizer.decode(next_token[0]), end="", flush=True)
    input_ids = torch.cat([input_ids, next_token], dim=-1)
print("\n... stuck forever!")


--- The Greedy Trap ---
Prompt: 'A cat is a cat is a'

Tracking the next 5 tokens, their probabilities, and the distribution entropy:

[Step 1] Entropy: 6.9375
  1. ' cat': 0.2158 <-- Greedy Choice
  2. ' dog': 0.1226 
  3. ' mouse': 0.0659 

[Step 2] Entropy: 4.0312
  1. ' is': 0.4688 <-- Greedy Choice
  2. '.': 0.1523 
  3. ',': 0.0718 

[Step 3] Entropy: 2.4062
  1. ' a': 0.8047 <-- Greedy Choice
  2. ' what': 0.0115 
  3. ' the': 0.0095 

[Step 4] Entropy: 2.8906
  1. ' cat': 0.7578 <-- Greedy Choice
  2. ' dog': 0.0203 
  3. ' very': 0.0167 

[Step 5] Entropy: 3.7344
  1. ' is': 0.4727 <-- Greedy Choice
  2. '.': 0.1533 
  3. '\n': 0.0498 

--- Generating 30 more tokens to see the full loop ---
A cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is
... stuck forever!
